# Trabalho 2 - Combinação de Fatores para Seleção de Portifólios de Ações

**Aluno:** Luiz Fernando Rabelo (11796893)

## Bibliotecas Utilizadas

Para a resolução do trabalho, foram utilizadas as bibliotecas numpy, pandas e riskfolio, as quais são importadas abaixo:

In [ ]:
import numpy as np
import pandas as pd
import riskfolio

## Dados Considerados

Foram considerados dados de mercado e de fatores coletados pela plataforma Economatica. Os dados estão dispostos em arqivos xlsx na pasta `/data` e são lidos abaixo.

In [ ]:
# Inicialização de dicionário para os dados de mercado:
dados_mercado = {
    'Comp-IBRX': None,   # composição do índice IBX
    'Fechamento': None,  # preços de fechamento dos ativos
    'Base': None,        # índices de referência (Ibov, IBX, Selic, etc)
}

# Leitura dos dados de mercado dos arquivos:
for chave in dados_mercado.keys():
    dados_mercado[chave] = pd.read_excel(f'./data/Dados-{chave}.xlsx', engine='openpyxl')
    dados_mercado[chave].set_index(keys='Data', inplace=True)

# Inicialização de dicionário para os dados de fatores:
dados_fatores = {
    'ROIC-A2': None,     # fator qualidade
    'Momentum-12': None, # fator momentum
    'Val-Merc': None,    # fator tamanho 
    'PVP': None,         # fator valor
    'Vol-12': None       # fator volatilidade
}

# Leitura dos dados de fatores dos arquivos:
for chave in dados_fatores.keys():
    dados_fatores[chave] = pd.read_excel(f'./data/Dados-{chave}.xlsx', engine='openpyxl')
    dados_fatores[chave].set_index(keys='Data', inplace=True)
    crescente = False if chave in ['ROIC-A2', 'Momentum-12'] else True
    dados_fatores[chave] = dados_fatores[chave].rank(axis=1, numeric_only=True, ascending=crescente, method='first')

# Definição de constantes complementares:
linha_inicio_dados = 13                     # correspondente à 01/2005
linha_fim_dados = linha_inicio_dados + 192  # correspondente à 12/2020
colunas = len(dados_mercado['Comp-IBRX'].columns)
passos_portifolio = 1
passos_avaliacao = 1

# Impressão dos dados de avaliação / rebalanceamento:
print(f'Período de avaliação - de: {dados_mercado["Base"].index[linha_inicio_dados]} ({linha_inicio_dados}) até: {dados_mercado["Base"].index[linha_fim_dados - 1]} ({linha_fim_dados - 1})')
print(f'Rebalanceamento a cada {passos_avaliacao}/{passos_portifolio} meses.')

## Definição de Funções Auxiliares

Com referência ao material utilizado nas aulas teóricas, foram aproveitadas as seguintes funções, que são usadas invocadas no decorrer do trabalho:

#### Função para Seleção de Portifólio por Fator

Dadas as informações que descrevem o fator e dada a fatia do ranking considerada (o ranking deverá estar ordenado de forma que a primeira posição é a melhor e a última a pior), a função retorna um dataframe com as ações selecionadas para o portifólio, separando-as por mês.


In [ ]:
def seleciona_port_1_fator(portifolio_original, inicio, fim):
    portifolio_final = portifolio_original.copy()
    portifolio_final.loc[:, :] = 0
    for lin in range(linha_inicio_dados, linha_fim_dados, passos_portifolio):
        for col in range(colunas):
            if inicio <= portifolio_original.iat[lin-1, col] <= fim:
                portifolio_final.iat[lin-1, col] = 1    
    return portifolio_final

#### Função para Intersecção de Portifólios

Funciona de forma semelhante à primeira função, mas considera a intersecção dos ranking de 2 fatores (n primeiras ações do primeiro + n primeiras ações do segundo) para a formação do portifólio. Retorna um dataframe com as ações selecionadas, separando-as por mês.

In [ ]:
def seleciona_port_2_fatores(portifolio1, fim1, portifolio2, fim2):
    portifolio_final = portifolio1.copy()
    portifolio_final.loc[:, :] = 0
    for lin in range(linha_inicio_dados, linha_fim_dados, passos_portifolio):
        for col in range(colunas):
            if 1 <= portifolio1.iat[lin-1, col] <= fim1 and 1 <= portifolio2.iat[lin-1, col] <= fim2:
                portifolio_final.iat[lin-1, col] = 1                
    return portifolio_final

#### Função para Avaliação de Portifólio

Recebe o portifólio montado e os preços de fechamento e retorna vários dados sobre o portifólio: lista de retorno acumulado, lista com retornos periódicos, lista com drawdown e retorno e volatilidade anualizados.

In [ ]:
def avalia_portifolio(portifolio, fechamentos):
    retornos_acumulados = [1]
    retornos_periodicos = []
    drawdowns = []
    retorno_acumulado = 1.0
    custo_trans = 0.0006
    
    for lin in range(linha_inicio_dados, linha_fim_dados, passos_avaliacao):
        contagem_ativos = 0.0
        rentabilidade_ativos = 0.0
        for col in range(colunas):
            if portifolio.iat[lin-1, col] > 0 and fechamentos.iat[lin-1, col] > 0 and fechamentos.iat[lin-1+passos_avaliacao, col] > 0:
                rentabilidade_ativos += (fechamentos.iat[lin-1+passos_avaliacao,col] / fechamentos.iat[lin-1,col]-1) * (portifolio.iat[lin-1, col])
                contagem_ativos += portifolio.iat[lin-1, col]
        if contagem_ativos == 0:
        #   return [1,1], [1,1], [0,0], 0, 0.000001
            return None
        retorno_acumulado *= (1.0 + rentabilidade_ativos / contagem_ativos - custo_trans)
        retornos_periodicos.append(rentabilidade_ativos / contagem_ativos - custo_trans)
        retornos_acumulados.append(retorno_acumulado)
        drawdowns.append(retorno_acumulado / np.max(retornos_acumulados) - 1)

    retorno_anualizado = pow(retorno_acumulado, 12 / (linha_fim_dados - linha_inicio_dados)) - 1
    volatilidade_anualizada = np.std(retornos_periodicos) * ((12 / passos_avaliacao) ** (1/2))

    return {
        'ret_accs': retornos_acumulados,
        'variacoes': retornos_periodicos,
        'draw_downs': drawdowns,
        'ret_aa': retorno_anualizado,
        'vol_aa': volatilidade_anualizada
    }


#### Função para Avaliação de Referência

Recebe a referência a ser considerada e retorna vários dados sobre a mesma: lista de retorno acumulado, lista com retornos periódicos, lista com drawdown e retorno e volatilidade anualizados.

In [ ]:
INDICE_IBOVESPA = 0
INDICE_IBX = 1
INDICE_SELIC = 2
INDICE_IPCA = 3

def avalia_referencia(referencia, indice):
    retornos_acumulados = [1]
    retornos_periodicos = []
    drawdowns = []
    retorno_acumulado = 1.0

    for lin in range(linha_inicio_dados, linha_fim_dados, passos_avaliacao):
        rentabilidade = referencia.iat[lin - 1 + passos_avaliacao, indice] / referencia.iat[lin - 1, indice]
        retorno_acumulado *= rentabilidade
        retornos_periodicos.append(rentabilidade-1)
        retornos_acumulados.append(retorno_acumulado)
        drawdowns.append(retorno_acumulado / np.max(retornos_acumulados) - 1)
    
    retorno_anualizado = pow(retorno_acumulado, 12 / (linha_fim_dados - linha_inicio_dados)) - 1
    volatilidade_anualizada = np.std(retornos_periodicos) * ((12 / passos_avaliacao) ** (1/2))

    return {
        'ret_accs': retornos_acumulados,
        'variacoes': retornos_periodicos,
        'draw_downs': drawdowns,
        'ret_aa': retorno_anualizado,
        'vol_aa': volatilidade_anualizada
    }

#### Função de Otimização do Riskfolio

In [ ]:
def seleciona_portifolio_otimizado(portifolio, tipo_otimizacao, fechamentos):
    total_meses = 24  # total de meses p/ cálculo da volatilidade
    portifolio_copia = portifolio.copy()

    for lin in range(linha_inicio_dados + total_meses, linha_fim_dados):
        df = pd.DataFrame()
        for col in range(colunas):
            if portifolio_copia.iat[lin-1, col] > 0:
                df[portifolio_copia.columns[col]] = fechamentos[portifolio_copia.columns[col]].iloc[lin-1-total_meses:lin-1]
        df.fillna(method='backfill', axis=0, inplace=True)
        df_mudancas = df.pct_change().dropna()

        if tipo_otimizacao == 'RP':  # risk parity
            portifolio_rp = riskfolio.Portfolio(returns=df_mudancas)
            portifolio_rp.assets_stats(d=0.94, method_cov='hist')
            dados_pesos = portifolio_rp.rp_optimization(rm='MV', b=None)
        elif tipo_otimizacao == 'GMV':  # minimum variance
            portifolio_gmv = riskfolio.Portfolio(returns=df_mudancas)
            portifolio_gmv.assets_stats(d=0.94)
            dados_pesos = portifolio_gmv.optimization(model='Classic', rm='MV', obj='MinRisk')
        elif tipo_otimizacao == 'MDP':  # maximum decorrelation
            portifolio_mdp = riskfolio.Portfolio(returns=df_mudancas)
            portifolio_mdp.assets_stats(d=0.94)
            portifolio_mdp.cov = df_mudancas.corr()
            dados_pesos = portifolio_mdp.optimization(model='Classic', rm='MV', obj='MinRisk')

        tamanho_colunas = len(df_mudancas.columns) 
        for at in range(tamanho_colunas):
            portifolio_copia.at[portifolio_copia.index[lin-1], df.columns[at]] = dados_pesos['weights'].iat[at]
    
    return portifolio_copia.copy()
            

## Cálculo da Rentabilidade, Volatilidade e Drawdown de Índices

A fins de comparação foram avaliados os desempenhos da taxa selic e índices Ibovespa e IBX.

In [ ]:
indices = {'SELIC': INDICE_SELIC, 'Ibov': INDICE_IBOVESPA, 'IBX': INDICE_IBX}
dados_indices = {}

for chave in indices.keys():
    dados_idc = avalia_referencia(dados_mercado['Base'], indices[chave])
    dados_indices[chave] = dados_idc
    print(f'Referência: {chave}')
    print(f'- Retorno acumulado: {round(dados_idc["ret_accs"][-1] * 100-100, 2)}%')
    print(f'- Retorno anualizado: {round(dados_idc["ret_aa"] * 100, 2)}%')
    print(f'- Volatilidade anualizada: {round(dados_idc["vol_aa"] * 100, 2)}%')
    print(f'- Relação Retorno/Volatilidade: {round(dados_idc["ret_aa"] / dados_idc["vol_aa"], 2)}')
    print(f'- Draw Down: {round(np.min(dados_idc["draw_downs"]) * 100, 2)}%\n')

## Avaliação de Portifólios Individuais (por Fator)

Podemos analisar cada fator, de forma individual, para compor os ativos de um portifólio. Nesse sentido, na célula abaixo estão dispostos os dados coletados por backtesting:

In [ ]:
dados_portifolios = {}

for chave in dados_fatores.keys():
    portifolio = seleciona_port_1_fator(dados_fatores[chave], 0, 30)
    dados_port = avalia_portifolio(portifolio, dados_mercado['Fechamento'])
    dados_portifolios[chave] = dados_port
    beta, alpha = np.polyfit(dados_indices['IBX']['variacoes'], dados_port['variacoes'], 1)
    print(f'Fator considerado: {chave}')
    print(f'- Retorno acumulado: {round(dados_port["ret_accs"][-1] * 100-100, 2)}%')
    print(f'- Retorno anualizado: {round(dados_port["ret_aa"] * 100, 2)}%')
    print(f'- Volatilidade anualizada: {round(dados_port["vol_aa"] * 100, 2)}%')
    print(f'- Relação Retorno/Volatilidade: {round(dados_port["ret_aa"] / dados_port["vol_aa"], 2)}')
    print(f'- Draw Down: {round(np.min(dados_port["draw_downs"]) * 100, 2)}%')
    print(f'- Alfa: {round(alpha * (12 / passos_avaliacao) * 100, 2)}%, Beta: {round(beta,2)}\n')

### Análise da Rentabilidade

In [ ]:
# Evolução das cotas de cada portifólio comparadas com o índice de referência IBX:
df = pd.DataFrame(index=dados_fatores['ROIC-A2'].iloc[linha_inicio_dados:linha_fim_dados+1].index)
for chave in dados_fatores.keys():
    df[chave] = dados_portifolios[chave]['ret_accs']
df['IBX'] = dados_indices['IBX']['ret_accs']
df.iloc[0:].plot(figsize=(18,6), grid=True, title='Evolução das cotas e índice');

Como se pode observar, o ranking de fatores de maior rentabilidade acumulada, considerando as 30 melhores ações de cada fator no período considerado, é dado por:

1. Fator Momentum;
2. Fator Volatilidade;
3. Fator Qualidade (ROIC);
4. Fator Preço (PVP);
5. Fator Tamanho (Valor de Mercado).

### Análise da Volatilidade

In [ ]:
# Evolução da volatilidade dos últimos 12 meses de cada portfólio comparadas com o índice de referência IBX:
df = pd.DataFrame()
for chave in dados_fatores.keys():
    df[chave] = pd \
        .Series(dados_portifolios[chave]['variacoes']) \
        .rolling(int(12 / passos_avaliacao)) \
        .std() * (int(12 / passos_avaliacao) ** (1/2))
df['IBX'] = pd \
    .Series(dados_indices['IBX']['variacoes']) \
    .rolling(int(12/passos_avaliacao)) \
    .std()*(int(12/passos_avaliacao)**(1/2))
df.plot(figsize=(18,6), grid=True, title='Volatilidade nos últimos 12 meses');

Já com relação à menor volatilidade, considerando-se as 30 melhores ações de cada fator, o ranking de fatores é dado por:

1. Fator Volatilidade;
2. Fator Qualidade (ROIC);
3. Fator Momentum;
4. Fator Preço (PVP);
5. Fator Tamanho (Valor de Mercado).

## Composição dos Portifólios

### Portifólio com Foco em Rentabilidade (Composição Intuitiva)

Para compor o portifólio "mais agressivo", com foco em rentabilidade, podemos considerar intuitivamente, a análise anterior (30 melhores ativos por fator) e usar os 2 fatores que apresentaram melhor desempenho individual na rentabilidade acumulada: _Momentum_ e _Volatilidade_.

In [ ]:
portifolio_rentabilidade = seleciona_port_2_fatores(dados_fatores['Momentum-12'], 30, dados_fatores['Vol-12'], 30)
dados_port = avalia_portifolio(portifolio_rentabilidade, dados_mercado['Fechamento'])
beta, alpha = np.polyfit(dados_indices['IBX']['variacoes'], dados_port['variacoes'], 1)

print(f'Fatores Momentum e Volatilidade')
print(f'- Retorno acumulado: {round(dados_port["ret_accs"][-1] * 100-100, 2)}%')
print(f'- Retorno anualizado: {round(dados_port["ret_aa"] * 100, 2)}%')
print(f'- Volatilidade anualizada: {round(dados_port["vol_aa"] * 100, 2)}%')
print(f'- Relação Retorno/Volatilidade: {round(dados_port["ret_aa"] / dados_port["vol_aa"], 2)}')
print(f'- Draw Down: {round(np.min(dados_port["draw_downs"]) * 100, 2)}%')
print(f'- Alfa: {round(alpha * (12 / passos_avaliacao) * 100, 2)}%, Beta: {round(beta,2)}\n')

In [ ]:
# Evolução das cotas do portifólio comparadas com o índice de referência IBX:
df = pd.DataFrame(index=dados_fatores['ROIC-A2'].iloc[linha_inicio_dados:linha_fim_dados+1].index)
df['Momentum & Volatilidade'] = dados_port['ret_accs']
df['IBX'] = dados_indices['IBX']['ret_accs']
df.iloc[0:].plot(figsize=(18,6), grid=True, title='Evolução das cotas e índice');

Percebe-se que o retorno acumulado do portifólio é bastante superior ao retorno acumulado do _IBX_.

In [ ]:
# Retorno anual de um portfólio comparado à referência IBX
df2 = pd.DataFrame(columns=['Data', 'Momentum & Volatilidade', 'IBX'])
for ind in range(0, len(df.index) - 12, 12):
    tempo = df.iloc[ind+12] / df.iloc[ind] - 1
    df2 = df2.append(tempo, ignore_index=True)
    df2.iat[len(df2) - 1, 0] = df.index[ind]
df2.set_index(keys='Data', inplace=True)
df2.plot.bar(figsize=(18,6), grid=True);

### Portifólio com Foco em Rentabilidade (Composição Intuitiva Otimizada)

É possível, ainda, distribuir os pesos de alocação considerando os métodos de otimização da biblioteca _Riskfolio_: 

- RP: Risk Parity;
- GMV: Minimum Variance;
- MDP: Maximum Decorrelation.

In [ ]:
labels_otimizacao = ['RP', 'GMV', 'MDP']

for label in labels_otimizacao:
    portifolio_rent_otimizado = seleciona_portifolio_otimizado(portifolio_rentabilidade, label, dados_mercado['Fechamento'])
    dados_port = avalia_portifolio(portifolio_rent_otimizado, dados_mercado['Fechamento'])
    beta, alpha = np.polyfit(dados_indices['IBX']['variacoes'], dados_port['variacoes'], 1)
    print(f'Fatores Momentum e Volatilidade - Otimização {label}')
    print(f'- Retorno acumulado: {round(dados_port["ret_accs"][-1] * 100-100, 2)}%')
    print(f'- Retorno anualizado: {round(dados_port["ret_aa"] * 100, 2)}%')
    print(f'- Volatilidade anualizada: {round(dados_port["vol_aa"] * 100, 2)}%')
    print(f'- Relação Retorno/Volatilidade: {round(dados_port["ret_aa"] / dados_port["vol_aa"], 2)}')
    print(f'- Draw Down: {round(np.min(dados_port["draw_downs"]) * 100, 2)}%')
    print(f'- Alfa: {round(alpha * (12 / passos_avaliacao) * 100, 2)}%, Beta: {round(beta,2)}\n')

Percebe-se que, com o portifólio otimizado, a rentabilidade acumulada foi reduzida para todos os métodos de otimização. Entretanto os valores de volatilidades anualizadas foram parecidos com os do portifólio original e os valores de draw down foram relativamente piores. A otimização mais adequada para o foco em rentabilidade, nesse caso, é a _RP_, uma vez que tem a maior rentabilidade acumulada.

In [ ]:
# Otimização RP:
portifolio_rp = seleciona_portifolio_otimizado(portifolio_rentabilidade, 'RP', dados_mercado['Fechamento'])
dados_port = avalia_portifolio(portifolio_rp, dados_mercado['Fechamento'])

# Evolução das cotas do portifólio comparadas com o índice de referência IBX:
df = pd.DataFrame(index=dados_fatores['ROIC-A2'].iloc[linha_inicio_dados:linha_fim_dados+1].index)
df['Momentum & Volatilidade - Otimização RP'] = dados_port['ret_accs']
df['IBX'] = dados_indices['IBX']['ret_accs']
df.iloc[0:].plot(figsize=(18,6), grid=True, title='Evolução das cotas e índice');

In [ ]:
# Retorno anual de um portfólio comparado à referência IBX
df2 = pd.DataFrame(columns=['Data', 'Momentum & Volatilidade - Otimização RP', 'IBX'])
for ind in range(0, len(df.index) - 12, 12):
    tempo = df.iloc[ind+12] / df.iloc[ind] - 1
    df2 = df2.append(tempo, ignore_index=True)
    df2.iat[len(df2) - 1, 0] = df.index[ind]
df2.set_index(keys='Data', inplace=True)
df2.plot.bar(figsize=(18,6), grid=True);

### Portifólio com Foco em Rentabilidade (Composição com Teste de Variação de Parâmetros Otimizada)

No intuito de testar diferentes fatores e alterar a quantidade de ativos alocados sobre cada fator, foi desenvolvido o seguinte teste com dados passados:

```
  para cada par de fatores:
    para cada quantidade de ativos do fator 1 no intervalo [15,50] (pulando de 5 em 5)
      para cada quantidade de ativos do fator 2 no intervalo [15,50] (pulando de 5 em 5)
        se as quantidades supracitadas atuais produzem otimização válida pela biblioteca Riskfolio:
          verificar rentabilidade das otimizações considerando as quantidades atuais
```

O resultado encontrado foi que, embora os fatores Momentum e Volatilidade tenham tido os melhores desempenhos individuais para 30 ações, os fatores Momentum e Qualidade com 25 e 40 ações, respectivamente, obtiveram um desempenho consideravelmente melhor para a rentabilidade quando otimizado pelo método Maximum Decorrelation:

In [ ]:
# Composição do portifólio:
portifolio_rentabilidade2 = seleciona_port_2_fatores(dados_fatores['ROIC-A2'], 40, dados_fatores['Momentum-12'], 25)

# Otimização do portifólio:
portifolio_rent_otimizado2 = seleciona_portifolio_otimizado(portifolio_rentabilidade2, 'MDP', dados_mercado['Fechamento'])
dados_port = avalia_portifolio(portifolio_rent_otimizado2, dados_mercado['Fechamento'])
beta, alpha = np.polyfit(dados_indices['IBX']['variacoes'], dados_port['variacoes'], 1)

# Exibição de resultados:
print(f'Fatores Qualidade e Momentum - Otimização MDP')
print(f'- Retorno acumulado: {round(dados_port["ret_accs"][-1] * 100-100, 2)}%')
print(f'- Retorno anualizado: {round(dados_port["ret_aa"] * 100, 2)}%')
print(f'- Volatilidade anualizada: {round(dados_port["vol_aa"] * 100, 2)}%')
print(f'- Relação Retorno/Volatilidade: {round(dados_port["ret_aa"] / dados_port["vol_aa"], 2)}')
print(f'- Draw Down: {round(np.min(dados_port["draw_downs"]) * 100, 2)}%')
print(f'- Alfa: {round(alpha * (12 / passos_avaliacao) * 100, 2)}%, Beta: {round(beta,2)}\n')

In [ ]:
# Evolução das cotas do portifólio comparadas com o índice de referência IBX:
df = pd.DataFrame(index=dados_fatores['ROIC-A2'].iloc[linha_inicio_dados:linha_fim_dados+1].index)
df['Momentum & Qualidade - Otimização MDP'] = dados_port['ret_accs']
df['IBX'] = dados_indices['IBX']['ret_accs']
df.iloc[0:].plot(figsize=(18,6), grid=True, title='Evolução das cotas e índice');

In [ ]:
# Retorno anual de um portfólio comparado à referência IBX
df2 = pd.DataFrame(columns=['Data', 'Momentum & Qualidade - Otimização MDP', 'IBX'])
for ind in range(0, len(df.index) - 12, 12):
    tempo = df.iloc[ind+12] / df.iloc[ind] - 1
    df2 = df2.append(tempo, ignore_index=True)
    df2.iat[len(df2) - 1, 0] = df.index[ind]
df2.set_index(keys='Data', inplace=True)
df2.plot.bar(figsize=(18,6), grid=True);

### Portifólio com Foco em Volatilidade (Composição Intuitiva)

Para compor o portifólio "mais moderado", com foco em menor volatilidade, podemos usar, inicialmente, a mesma ideia intuitiva de considerar a análise dos 30 melhores ativos por fator e usar os 2 fatores que apresentaram melhor desempenho individual na volatilidade: _Volatilidade_ e _Qualidade_.

In [ ]:
portifolio_volatilidade = seleciona_port_2_fatores(dados_fatores['Vol-12'], 30, dados_fatores['ROIC-A2'], 30)
dados_port = avalia_portifolio(portifolio_volatilidade, dados_mercado['Fechamento'])
beta, alpha = np.polyfit(dados_indices['IBX']['variacoes'], dados_port['variacoes'], 1)

print(f'Fatores Volatilidade e Qualidade')
print(f'- Retorno acumulado: {round(dados_port["ret_accs"][-1] * 100-100, 2)}%')
print(f'- Retorno anualizado: {round(dados_port["ret_aa"] * 100, 2)}%')
print(f'- Volatilidade anualizada: {round(dados_port["vol_aa"] * 100, 2)}%')
print(f'- Relação Retorno/Volatilidade: {round(dados_port["ret_aa"] / dados_port["vol_aa"], 2)}')
print(f'- Draw Down: {round(np.min(dados_port["draw_downs"]) * 100, 2)}%')
print(f'- Alfa: {round(alpha * (12 / passos_avaliacao) * 100, 2)}%, Beta: {round(beta,2)}\n')

In [ ]:
# Evolução da volatilidade dos últimos 12 meses de cada portfólio comparadas com o índice de referência IBX:
df = pd.DataFrame()
df['Volatilidade & Qualidade'] = pd \
    .Series(dados_port['variacoes']) \
    .rolling(int(12 / passos_avaliacao)) \
    .std() * (int(12 / passos_avaliacao) ** (1/2))
df['IBX'] = pd \
    .Series(dados_indices['IBX']['variacoes']) \
    .rolling(int(12 / passos_avaliacao)) \
    .std()*(int(12 / passos_avaliacao) ** (1/2))
df.plot(figsize=(18,6), grid=True, title='Volatilidade nos últimos 12 meses');

Percebe-se no resultado, a volatilidade é consideravelmente menor em relação ao índice IBX.

In [ ]:
# Retorno anual de um portfólio comparado à referência IBX
df2 = pd.DataFrame(columns=['Data', 'Volatilidade & Qualidade', 'IBX'])
for ind in range(0, len(df.index) - 12, 12):
    tempo = df.iloc[ind+12] / df.iloc[ind] - 1
    df2 = df2.append(tempo, ignore_index=True)
    df2.iat[len(df2) - 1, 0] = df.index[ind]
df2.set_index(keys='Data', inplace=True)
df2.plot.bar(figsize=(18,6), grid=True);

### Portifólio com Foco em Volatilidade (Composição Intuitiva Otimizada)

Otimizando o portifólio com os métodos do _Riskfolio_, obtemos:

In [ ]:
labels_otimizacao = ['RP', 'GMV', 'MDP']

for label in labels_otimizacao:
    portifolio_vol_otimizado = seleciona_portifolio_otimizado(portifolio_volatilidade, label, dados_mercado['Fechamento'])
    dados_port = avalia_portifolio(portifolio_vol_otimizado, dados_mercado['Fechamento'])
    beta, alpha = np.polyfit(dados_indices['IBX']['variacoes'], dados_port['variacoes'], 1)
    print(f'Fatores Volatilidade e Qualidade - Otimização {label}')
    print(f'- Retorno acumulado: {round(dados_port["ret_accs"][-1] * 100-100, 2)}%')
    print(f'- Retorno anualizado: {round(dados_port["ret_aa"] * 100, 2)}%')
    print(f'- Volatilidade anualizada: {round(dados_port["vol_aa"] * 100, 2)}%')
    print(f'- Relação Retorno/Volatilidade: {round(dados_port["ret_aa"] / dados_port["vol_aa"], 2)}')
    print(f'- Draw Down: {round(np.min(dados_port["draw_downs"]) * 100, 2)}%')
    print(f'- Alfa: {round(alpha * (12 / passos_avaliacao) * 100, 2)}%, Beta: {round(beta,2)}\n')

A otimização reduziu um pouco mais a volatilidade para o método GMV.

In [ ]:
portifolio_gmv = seleciona_portifolio_otimizado(portifolio_volatilidade, 'GMV', dados_mercado['Fechamento'])
dados_port = avalia_portifolio(portifolio_gmv, dados_mercado['Fechamento'])

# Evolução da volatilidade dos últimos 12 meses de cada portfólio comparadas com o índice de referência IBX:
df = pd.DataFrame()
df['Volatilidade & Qualidade - Otimização GMV'] = pd \
    .Series(dados_port['variacoes']) \
    .rolling(int(12 / passos_avaliacao)) \
    .std() * (int(12 / passos_avaliacao) ** (1/2))
df['IBX'] = pd \
    .Series(dados_indices['IBX']['variacoes']) \
    .rolling(int(12 / passos_avaliacao)) \
    .std()*(int(12 / passos_avaliacao) ** (1/2))
df.plot(figsize=(18,6), grid=True, title='Volatilidade nos últimos 12 meses');

In [ ]:
# Retorno anual de um portfólio comparado à referência IBX
df2 = pd.DataFrame(columns=['Data', 'Volatilidade & Qualidade - Otimização GMV', 'IBX'])
for ind in range(0, len(df.index) - 12, 12):
    tempo = df.iloc[ind+12] / df.iloc[ind] - 1
    df2 = df2.append(tempo, ignore_index=True)
    df2.iat[len(df2) - 1, 0] = df.index[ind]
df2.set_index(keys='Data', inplace=True)
df2.plot.bar(figsize=(18,6), grid=True);